In [3]:
import cvxpy as cp
x = cp.Variable(1)
objective = cp.Minimize(x ** 2 - 6 * x + 10)
constraints = [x >= 4, x <= 10]
prob = cp.Problem(objective, constraints)
result = prob.solve()
x.value

# Non-convex function would return: DCPError: Problem does not follow DCP rules.
# DCP stands for Disciplined Convex Programming

array([4.])

In [16]:
# Mean variance optimization using cvxpy library

import numpy as np
Sigma = np.matrix([[0.0225 , 0.0216 , 0.00075],   
                   [0.0216 , 0.0324 , 0.00045], 
                   [0.00075, 0.00045, 0.0025]])
mu = np.array([.06, .05, .03])
mu_p = .055
N = len(mu)
w = cp.Variable(N)
objective = cp.Minimize(cp.quad_form(w, Sigma))
constraints = [w.T @ mu >= mu_p, cp.sum(w) == 1]
prob = cp.Problem(objective, constraints)
result = prob.solve()
print("Long/short portfolio: " + str(w.value))

# Variant with long only portfolio
constraints = [w.T @ mu >= mu_p, cp.sum(w) == 1, w >= 0]
prob = cp.Problem(objective, constraints)
result = prob.solve()
print("Long only portfolio: " + str(w.value))

# Variant constraining risk
sigma_b = .10
N = len(mu)
w = cp.Variable(N)
objective = cp.Maximize(mu.T @ w)
constraints = [cp.quad_form(w, Sigma) <= sigma_b ** 2, cp.sum(w) == 1]
prob = cp.Problem(objective, constraints)
result = prob.solve()
print("Risk constraint portfolio: " + str(w.value))

Long/short portfolio: [ 1.0798722  -0.36980831  0.2899361 ]
Long only portfolio: [8.33333333e-01 3.92339400e-24 1.66666667e-01]
Risk constraint portfolio: [ 0.88354576 -0.29400255  0.41045678]


In [17]:
import pandas as pd

def get_default_inputs():
    tickers = ['VTI', 'VEA', 'VWO', 'AGG', 'BNDX', 'EMB']
    ers = pd.Series([.05, .05, .07, .03, .02, .04], tickers)
    sigma = np.array(
        [[0.0287, 0.0250, 0.0267, 0.0000, 0.0002, 0.0084],
         [0.0250, 0.0281, 0.0288, 0.0003, 0.0002, 0.0092],
         [0.0267, 0.0288, 0.0414, 0.0005, 0.0004, 0.0112],
         [0.0000, 0.0003, 0.0005, 0.0017, 0.0008, 0.0019],
         [0.0002, 0.0002, 0.0004, 0.0008, 0.0010, 0.0011],
         [0.0084, 0.0092, 0.0112, 0.0019, 0.0011, 0.0083]])
    sigma = pd.DataFrame(sigma, tickers, tickers)
 
    return ers, sigma

from typing import Dict 
class Constraint:
 
    def generate_constraint(self, variables: Dict):
        """ Create the cvxpy Constraint
 
        :param variables: dictionary containing the cvxpy Variables for the
          problem
        :return: A cvxpy Constraint object representing the constraint
        """
        pass

class LongOnlyConstraint(Constraint):
 
    def __init__(self):
        """ Constraint to enforce all portfolio weights are non-negative
        """
        pass
 
    def generate_constraint(self, variables: Dict):
        return variables['w'] >= 0
 
 
class FullInvestmentConstraint(Constraint):
 
    def __init__(self):
        """ Constraint to enforce the sum of the portfolio weights is one
        """
        pass
 
    def generate_constraint(self, variables: Dict):
        return cp.sum(variables['w']) == 1.0

from typing import Union, List
import pandas as pd 
class TrackingErrorConstraint(Constraint):
 
    def __init__(self,
                 asset_names: Union[List[str], pd.Index],
                 reference_weights: pd.Series,
                 sigma: pd.DataFrame,
                 upper_bound: float):
        """ Constraint on the tracking error between a subset of the
        portfolio and a set of target weights
 
        :param asset_names: Names of all assets in the problem
        :param reference_weights: Vector of target weights. Index should be
          a subset of asset_names
        :param sigma: Covariance matrix, indexed by asset_names
        :param upper_bound: Upper bound for the constraint, in units of
          volatility (standard deviation)
        """
        self.reference_weights = \
            reference_weights.reindex(asset_names).fillna(0)
        self.sigma = sigma
        self.upper_bound = upper_bound ** 2
 
    def generate_constraint(self, variables: Dict):
        w = variables['w']
        tv = cp.quad_form(w - self.reference_weights, self.sigma)
        return tv <= self.upper_bound

class VolatilityConstraint(TrackingErrorConstraint):
 
    def __init__(self,
                 asset_names: Union[List[str], pd.Index],
                 sigma: pd.DataFrame,
                 upper_bound: float):
        """ Constraint on the overall volatility of the portfolio
 
        :param asset_names: Names of all assets in the problem
        :param sigma: Covariance matrix, indexed by asset_names
        :param upper_bound: Upper bound for the constraint, in units of
          volatility (standard deviation)
        """
 
        zeros = pd.Series(np.zeros(len(asset_names)), asset_names)
        super(VolatilityConstraint, self).__init__(asset_names, zeros,
                                                   sigma, upper_bound)

class MeanVarianceOpt:

    def __init__(self):
        self.asset_names = []
        self.variables = None
        self.prob = None
 
    @staticmethod
    def _generate_constraints(variables: Dict,
                              constraints: List[Constraint]):
        return [c.generate_constraint(variables) for c in constraints]
 
    def solve(self):
        self.prob.solve()
 
    def get_var(self, var_name: str):
        return pd.Series(self.variables[var_name].value, self.asset_names)
 
class MaxExpectedReturnOpt(MeanVarianceOpt):
 
    def __init__(self,
                 asset_names: Union[List[str], pd.Index],
                 constraints: List[Constraint],
                 ers: pd.Series):
        super().__init__()
        self.asset_names = asset_names
        variables = dict({'w': cp.Variable(len(ers))})
 
        cons = MeanVarianceOpt._generate_constraints(variables,
                                                     constraints)
        obj = cp.Maximize(ers.values.T @ variables['w'])
        self.variables = variables
        self.prob = cp.Problem(obj, cons)

In [18]:
# Detailed explanation: https://livebook.manning.com/book/build-a-robo-advisor-with-python-from-scratch/chapter-10/v-9/148
ers, sigma = get_default_inputs()
cons = [LongOnlyConstraint(), FullInvestmentConstraint(),
        VolatilityConstraint(ers.index, sigma, .15)]
o = MaxExpectedReturnOpt(ers.index, cons, ers)
o.solve()
weights = np.round(o.get_var('w'), 6)
print(weights)

VTI     0.000000
VEA     0.000000
VWO     0.731976
AGG     0.268022
BNDX    0.000000
EMB     0.000002
dtype: float64


In [19]:
# Perturbing the expected returns results in a wideley different portfolio

ers, sigma = get_default_inputs()
ers['VWO'] -= .01 # increase the expected returns by 1%
ers['VTI'] += .01 # decrease the expected returns by 1%
cons = [LongOnlyConstraint(), FullInvestmentConstraint(),
        VolatilityConstraint(ers.index, sigma, .15)]
o = MaxExpectedReturnOpt(ers.index, cons, ers)
o.solve()
weights = np.round(o.get_var('w'), 6)
print(weights)

VTI     0.763500
VEA     0.000000
VWO     0.072595
AGG     0.000001
BNDX    0.000000
EMB     0.163903
dtype: float64
